In [ ]:
import glob

from potoo.plot import *
from potoo.util import *
import sklearn

from cache import *
from constants import *
from datasets import *
from features import *
from load import *
from sp14.model import *
from util import *
from viz import *

figsize('inline_short');

In [ ]:
load = Load()
recs = load.recs(
    # limit=30,  # XXX Faster dev
    datasets=[
        'peterson-field-guide',
        'recordings',
    ],
)
display(df_summary(recs), recs[:5])

[06:31:32.331] Load.metadata:in
  len(recs): 633
  len(recs) per dataset: {peterson-field-guide: 535, recordings: 98}
[                                                                                          ] | 0% Completed |  0.0s

[                                                                                          ] | 0% Completed |  0.1s

[##################                                                                        ] | 20% Completed |  0.2s

[#####################################                                                     ] | 41% Completed |  0.3s

[######################################################                                    ] | 60% Completed |  0.4s

[##########################################################################                ] | 82% Completed |  0.5s

[##########################################################################################] | 100% Completed |  0.6s

[06:31:33.027] Load.metadata:out
  sum(duration_h): 5.07
  sum(samples_mb): 768.0
  sum(samples_n): 402551191
  n_species: 55


,dataset,species,species_com_name,species_query,duration_s,samples_mb,samples_n,basename,species_longhand,id,path
dtype,object,category,category,object,float64,float64,int64,object,category,object,object
sizeof,51.6 KiB,11.4 KiB,11.9 KiB,43.8 KiB,9.9 KiB,9.9 KiB,9.9 KiB,55.7 KiB,12.3 KiB,74.6 KiB,77.1 KiB
len,633,633,633,633,633,633,633,633,633,633,633
count,633,633,633,633,633,633,633,633,633,633,633
nunique,2,55,55,64,537,537,537,633,55,633,633
mean,NaN,NaN,NaN,NaN,28.841,1.213,635941.850,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,32.301,1.358,712235.547,NaN,NaN,NaN,NaN
min,peterson-field-guide,SCQU,Scaled Quail,BEWR,2.74,0.115,60480,081845,Scaled Quail - SCQU,peterson-field-guide/amecro/audio/43215281 Caw (clear),peterson-field-guide/amecro/audio/43215281 Caw (clear).mp3
25%,peterson-field-guide,TUTI,Tufted Titmouse,bewwre,15.50,0.650,340992,43217721 Chick-a-dee Call,Tufted Titmouse - TUTI,peterson-field-guide/canwre/audio/43219271 Male Song (unusual),peterson-field-guide/canwre/audio/43219271 Male Song (unusual).mp3
50%,peterson-field-guide,WIWR,Winter Wren,comrav,21.40,0.901,472320,43219841 Churt,Winter Wren - WIWR,peterson-field-guide/marwre/audio/44412311 Chit,peterson-field-guide/marwre/audio/44412311 Chit.mp3


,dataset,species,species_com_name,species_query,duration_s,samples_mb,samples_n,basename,species_longhand,id,path
479,peterson-field-guide,SCQU,Scaled Quail,scaqua,12.30,0.517,271296,43186591 Kuk-curr Series,Scaled Quail - SCQU,peterson-field-guide/scaqua/audio/43186591 Kuk-curr Series,peterson-field-guide/scaqua/audio/43186591 Kuk-curr Series.mp3
485,peterson-field-guide,SCQU,Scaled Quail,scaqua,12.10,0.509,266688,43186661 Chip-seer,Scaled Quail - SCQU,peterson-field-guide/scaqua/audio/43186661 Chip-seer,peterson-field-guide/scaqua/audio/43186661 Chip-seer.mp3
486,peterson-field-guide,SCQU,Scaled Quail,scaqua,15.60,0.657,344448,43186671 Kuk,Scaled Quail - SCQU,peterson-field-guide/scaqua/audio/43186671 Kuk,peterson-field-guide/scaqua/audio/43186671 Kuk.mp3
487,peterson-field-guide,SCQU,Scaled Quail,scaqua,7.73,0.325,170496,43186681 Wing sounds & Kuk,Scaled Quail - SCQU,peterson-field-guide/scaqua/audio/43186681 Wing sounds & Kuk,peterson-field-guide/scaqua/audio/43186681 Wing sounds & Kuk.mp3
482,peterson-field-guide,SCQU,Scaled Quail,scaqua,10.10,0.423,221760,43186621 Zreek Series,Scaled Quail - SCQU,peterson-field-guide/scaqua/audio/43186621 Zreek Series,peterson-field-guide/scaqua/audio/43186621 Zreek Series.mp3


# Fit search

In [ ]:
# Load models
projection = Projection.load('peterson-v0-26bae1c')
features = projection.features
load = features.load

[06:31:33.225] Projection.load
  path: /Users/danb/hack/bubo/data/models/projection/peterson-v0-26bae1c.pkl


In [ ]:
# Add features
recs = features.transform(recs)
recs = projection.transform(recs)

[06:31:33.275] Features.spectros:in
  len(recs): 633
  len(recs) per dataset: {peterson-field-guide: 535, recordings: 98}
  sum(duration_h): 5.07
  sum(samples_mb): 768.0
  sum(samples_n): 402551191
[                                                                                          ] | 0% Completed |  0.0s

[                                                                                          ] | 0% Completed |  0.1s

[                                                                                          ] | 0% Completed |  0.2s

[                                                                                          ] | 0% Completed |  0.3s

[                                                                                          ] | 0% Completed |  0.4s

...

[######################################################                                    ] | 60% Completed |  0.7s

[######################################################                                    ] | 60% Completed |  0.8s

[########################################################################                  ] | 80% Completed |  0.9s

[########################################################################                  ] | 80% Completed |  1.0s

[##########################################################################################] | 100% Completed |  1.2s

In [ ]:
recs_eval = (recs
    [lambda df: df.dataset == 'peterson-field-guide']
    # [:10]  # Faster dev
    .reset_index(drop=True)
)
train_n, test_n = (len(recs_eval)+1)//2, len(recs_eval)//2
recs_train, recs_test = (recs_eval
    .pipe(sklearn.utils.shuffle, random_state=0)
    .sample(train_n + test_n, random_state=0)
    .pipe(lambda df: (
        df[:train_n],
        df[train_n : train_n + test_n],
    ))
)
log('params', **{
    'recs_eval': len(recs_eval),
    'recs_train': len(recs_train),
    'recs_test': len(recs_test),
})

[06:31:48.305] params
  recs_eval: 535
  recs_train: 268
  recs_test: 267


In [ ]:
from sp14.model import *
search = Search(
    # n_neighbors=3,
    n_neighbors=5,
    # n_neighbors=10,
    projection=projection,
)
search.fit(recs_train)

[06:31:48.340] Search.fit:in
  recs: 268
  (n, f*p): [268, 1500]


________________________________________________________________________________
[Memory] Calling sp14.model._fit...
_fit({ 'key': ( 55                             peterson-field-guide/belkin1/audio/43205241 Low Rattle & High Rattle
127                peterson-field-guide/bkcchi/audio/43217251 Whistled Song (plastic or incomplete)
468                                    peterson-field-guide/orcwar/audio/43227341 Song (one series)
479                          peterson-field-guide/orcwar/audio/43227311 Song (mostly single trills)
277                                                 peterson-field-guide/houwre/audio/43219421 Song
64                                    peterson-field-guide/amecro/audio/43215351 Caw (harsh) & Rasp
57                                        peterson-field-guide/amecro/audio/43215421 Gurg..., 
{ 'args': ( Search(projection=Projection(features=Features(load=Load(channels=1, sample_rate=22050, sample_width_bit=16, cache_audio=True), sample_rate=22050, f_min=1000, f

[06:32:34.484] Search.fit:out
  knn.get_params: {algorithm: auto, leaf_size: 30, metric: minkowski, metric_params: null, n_jobs: 1, n_neighbors: 5, p: 2, weights: uniform}
  knn.classes_: [SCQU, MONQ, WITU, OSPR, KILL, BEKI, AMCR, CORA, CACH, BCCH, BOCH, TUTI, BCTI, BUSH, RBNU, WBNU, BRCR, CACW, ROWR, CANW, CARW, BEWR, HOWR, WIWR, SEWR, MAWR, SWTH, HETH, AMRO, EUST, CEDW, OCWA, WIWA, SOSP, WCSP]
  knn.classes_.len: 35


Search(projection=Projection(features=Features(load=Load(channels=1, sample_rate=22050, sample_width_bit=16, cache_audio=True), sample_rate=22050, f_min=1000, f_bins=40, hop_length=256, frame_length=512, frame_window='hann', patch_length=4), skm_fit_max_t=600000, k=500, variance_explained=0.99, do_pca=True, pca_whiten=True, standardize=False, normalize=False, agg_funs=['mean', 'std', 'max']), n_neighbors=5)